# Space Pirates (Writeup)
###### _Categoría: Crypto_
###### _by: Danex_
###### _350 pts._
###### _Descarga los archivos [aquí](https://drive.google.com/file/d/1qScyu3OvYbqB8HPBtgICfNXxgNmu2Xkp/view?usp=sharing)_

Analizando el código, la función `create_pol(self)` realiza lo siguiente:
* Considerando que inicialmente `coeffs = [secret]`, se calculan los siguientes coeficientes del Shamir Secret Sharing (SSS) de la siguiente manera (no es necesario saber al detalle lo que hace el SSS, con el código tenemos suficiente para evidenciar el ataque): `coeffs[i] = md5(coeffs[i - 1])`
* Luego, se toman los $k$ primeros valores de `coeffs`
* Se generan las listas `x_vals` e `y_vals` de la siguiente manera:
    - `x_vals[i] = x`, donde `x` es un número aleatorio entre $1$ y $p - 1$
    - `y_vals[i]` $= \sum_{i = 0}^{k - 1}$ `coeffs[i]` $\cdot x ^ i$ mod $p$

Nuestro objetivo es obtener la variable `secret`, sabemos que está dentro de `coeffs`, pero los datos que nos entregan sobre el secreto compartido son: `x_vals[0]`, `y_vals[0]`, $p$, $k$, $n$ y `coeffs[1]`. Comencemos con la definición de `y_vals[0]` (llamémoslo desde ahora $y$):
\begin{equation}
y \equiv \sum_{i = 0}^{k - 1} c_i \cdot x^i \text{ mod } p
\end{equation}
donde $c_i = $ `coeffs[i]`. Desarrollando, se tiene (con $k = 10$):
\begin{equation}
y \equiv c_0 + c_1x + c_2x^2 + \ldots + c_9x^9 \text{ mod } p
\end{equation}
sabemos que $c_0 = $ `secret` y $c_i = \text{md5}(c_{i - 1})$. Además, pero tenemos los valores de $y$, $c_1$, $x$ y $p$. De esta forma, podemos despejar $c_0$ de la siguiente manera:
\begin{equation}
c_0 \equiv y - c_1x - \text{md5}(c_{1})x^2 - \ldots - \text{md5}(c_{8})x^9 \text{ mod } p
\end{equation}
donde el código que automatiza este proceso se encuentra a continuación:

In [ ]:
from hashlib import md5
y = 11086299714260406068
x = 21202245407317581090
coeff = 93526756371754197321930622219489764824
p = 92434467187580489687
k = 10
n = 18

def next_coeff(val):
    return int(md5(val.to_bytes(32, byteorder="big")).hexdigest(),16)
coeffs = [coeff]
for i in range(1, k - 1):
    coeffs.append(next_coeff(coeffs[i - 1]))

res = y
for i in range(len(coeffs)):
    res -= (coeffs[i] * x ** (i + 1)) % p
secret = res % p
secret

39612257993477957104

Al obtener el `secret = 39612257993477957104`, queda ingresarlo en un cifrado (a través de una `seed`) AES-ECB para obtener la flag.

_Ojito: antes de ejecutar el siguiente código, se requiere la descarga de algunas bibliotecas. Además, se requiere de Python 3.9 para utilizar la función `randbytes`, así que adjunto además el código [aquí](https://drive.google.com/file/d/1cV6YLTnIWQKmCzIJYKgIpo0NMmRKs6Rx/view?usp=sharing) (porque Colab usa el 3.6), la idea de este notebook es poder aprovechar el markdown para explicar los detalles del challenge_.

In [ ]:
pip install pycryptodome

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from random import randbytes, seed

enc_flag = '1aaad05f3f187bcbb3fb5c9e233ea339082062fc10a59604d96bcc38d0af92cd842ad7301b5b72bd5378265dae0bc1c1e9f09a90c97b35cfadbcfe259021ce495e9b91d29f563ae7d49b66296f15e7999c9e547fac6f1a2ee682579143da511475ea791d24b5df6affb33147d57718eaa5b1b578230d97f395c458fc2c9c36525db1ba7b1097ad8f5df079994b383b32695ed9a372ea9a0eb1c6c18b3d3d43bd2db598667ef4f80845424d6c75abc88b59ef7c119d505cd696ed01c65f374a0df3f331d7347052faab63f76f587400b6a6f8b718df1db9cebe46a4ec6529bc226627d39baca7716a4c11be6f884c371b08d87c9e432af58c030382b737b9bb63045268a18455b9f1c4011a984a818a5427231320ee7eca39bdfe175333341b7c'
seed(secret)
key = randbytes(16)
cipher = AES.new(key, AES.MODE_ECB)
flag = unpad(cipher.decrypt(bytes.fromhex(enc_flag)), 16)

donde se obtiene:  
`b'The treasure is located at galaxy VS-708.\nOur team needs 3 light years to reach it.\nOur solar cruise has its steam canons ready to fire in case we encounter enemies.\nNext time you will hear from us brother, everyone is going to be rich!\nHTB{1_d1dnt_kn0w_0n3_sh4r3_w45_3n0u9h!1337}'`
